In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# from cellpose import models
from cellpose.io import imread
import glob
from pathlib import Path
from PIL import Image, ImageSequence
from tqdm import tqdm
import os
import os.path
# from livecell_tracker import segment
from livecell_tracker import core
from livecell_tracker.core import datasets
from livecell_tracker.core.datasets import LiveCellImageDataset, SingleImageDataset
from skimage import measure
from livecell_tracker.core import SingleCellTrajectory, SingleCellStatic
# import detectron2
# from detectron2.utils.logger import setup_logger

# setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import cv2

Generate single cells from a mask dataset and write single cell json data to disk

In [ ]:
pos = "XY10"
dic_dir = "/home/ken67/LiveCellTracker-dev/datasets/tifs_CFP_A549_VIM_120hr_NoTreat_NA_YL_Ti2e_2023-03-22/XY10/DIC"
dic_dataset = LiveCellImageDataset(dir_path=dic_dir)

In [ ]:

# dataset_dir_path = Path(
#     f"../datasets/EBSS_Starvation/tif_STAV-A549_VIM_24hours_NoTreat_NA_YL_Ti2e_2022-12-21/{pos}/"
# )

# mask_dataset_path = Path(f"../datasets/EBSS_Starvation/tif_STAV-A549_VIM_24hours_NoTreat_NA_YL_Ti2e_2022-12-21/out/{pos}/seg")

# mask_dataset = LiveCellImageDataset(mask_dataset_path, ext="png")
# time2url = sorted(glob.glob(str((Path(dataset_dir_path) / Path("*_DIC.tif")))))
# time2url = {i: path for i, path in enumerate(time2url)}
# dic_dataset = LiveCellImageDataset(time2url=time2url, ext="tif")

# from livecell_tracker.segment.utils import prep_scs_from_mask_dataset
# single_cells = prep_scs_from_mask_dataset(mask_dataset, dic_dataset)

# scs_write_path = f"./datasets/test_scs_EBSS_starvation/{pos}/scs.json"
# dataset_dir = Path(f"./datasets/test_scs_EBSS_starvation/{pos}/datasets")
# SingleCellStatic.write_single_cells_json(single_cells, scs_write_path, dataset_dir=dataset_dir)


## Load model

In [ ]:
import torch
from pathlib import Path
# model_dir = Path(r"./scripts/mmdetection_classify/work_dirs/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v6-combined-clipL=2-CLIP_NUM=4/")
# model_dir = Path(r"./scripts/mmdetection_classify/work_dirs/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v8-combined-clipLen=2-trainClipNum=3-valClipNum=3")
# model_dir = Path(r"./scripts/mmdetection_classify/work_dirs/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v8-combined-clipLen=1-trainClipNum=3-valClipNum=3")
# path = r"./scripts/mmdetection_classify/work_dirs/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v6-combined-clipL=2/best_acc_top1_epoch_28.pth"
# model = torch.load(path)
# model_dir = Path(r"./scripts/mmdetection_classify/work_dirs/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v9-combined-clipLen=3-trainClipNum=3-valClipNum=3")
# model_dir = Path(r"./scripts/mmdetection_classify/work_dirs/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v10-st-combined-clipLen=3-trainClipNum=3-valClipNum=3")
# model_dir = Path(r"./scripts/mmdetection_classify/work_dirs/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v10-drop-div-combined-clipLen=2-trainClipNum=3-valClipNum=3")
model_dir = Path(r"./scripts/mmdetection_classify/work_dirs/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v10-st-combined-clipLen=2-trainClipNum=3-valClipNum=3")


Read a trained MMAction model


In [ ]:
from mmengine.config import Config, DictAction
from mmaction.registry import MODELS
config_file = str(model_dir / "tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb.py")
# checkpoint_file = str(model_dir / "best_acc_top1_epoch_5.pth")
# checkpoint_file = str(model_dir / "epoch_52.pth")
# checkpoint_file = str(model_dir / "best_acc_top1_epoch_126.pth")
# checkpoint_file = str(model_dir / "epoch_250.pth")
checkpoint_file = str(model_dir / "epoch_59.pth")
cfg = Config.fromfile(config_file)

In [ ]:
# model = MODELS.build()
import mmcv
from mmaction.apis import init_recognizer, inference_recognizer

DEVICE = 'cuda:0' # or 'cpu', based on your device
model = init_recognizer(config_file, checkpoint_file, device=DEVICE)

## Segmentation

In [ ]:
import cellpose.models 
pretrained_model_path = "/home/ken67/LiveCellTracker-dev/notebooks/notebook_results/cellpose/cellpose_A549_cyto2_cellbody/models/cellpose_residual_on_style_on_concatenation_off_cellpose_A549_cyto2_cellbody_2023_04_17_21_49_50.313712"
cp_model = cellpose.models.CellposeModel(pretrained_model=pretrained_model_path, gpu=True) #, model_type="cyto2")

preliminary check

In [ ]:
# TODO: move to pkg

from livecell_tracker.preprocess.utils import enhance_contrast, standard_preprocess
from livecell_tracker.segment.cellpose_utils import segment_single_images_by_cellpose, segment_single_image_by_cellpose
import random

num_img_to_viz = 5
times = dic_dataset.times
diameters = [30, 50, 80, 100]
for i in range(0, num_img_to_viz):
    # randomly sample a time from the dataset
    img = dic_dataset[times[random.randint(0, len(times) - 1)]]
    # img = normalize_img_to_uint8(img)
    img = standard_preprocess(img)
    masks = []
    for diameter in diameters:
        mask = segment_single_image_by_cellpose(img, cp_model, channels=[[0, 0]], diameter=diameter)
        masks.append(mask)

    # visualize
    fig_num = 1 + len(diameters)
    fig, axes = plt.subplots(1, fig_num, figsize=(fig_num * 5, 5))
    axes[0].imshow(enhance_contrast(img))
    axes[0].set_title("raw image")
    for i, mask in enumerate(masks):
        axes[i + 1].imshow(mask)
        axes[i + 1].set_title("diameter: {}".format(diameters[i]))
    plt.show()

In [ ]:
# from livecell_tracker.core.io_utils import save_png
# import tqdm
selected_diameter = 80
out_dir = Path(os.path.join('./notebook_results', f'CFP_{pos}_diameter={selected_diameter}'))
# mask_out_dir = out_dir / 'masks'
# mask_out_dir.mkdir(parents=True, exist_ok=True)
# filename = "mask_time-{}.png"
# times = dic_dataset.times
# for i in tqdm(range(0, len(dic_dataset))):
#     # randomly sample a time from the dataset
#     time = times[i]
#     img = dic_dataset[time]
#     # img = normalize_img_to_uint8(img)
#     img = standard_preprocess(img)
#     for diameter in diameters:
#         mask = segment_single_image_by_cellpose(img, cp_model, channels=[[0, 0]], diameter=diameter)
#         # save the mask
#         save_png(os.path.join(mask_out_dir, filename.format(int(time))), mask)



In [ ]:
mask_dataset_path = r"/home/ken67/LiveCellTracker-dev/notebooks/notebook_results/CFP_XY10_diameter=80/masks"
mask_dataset = LiveCellImageDataset(dir_path=mask_dataset_path, ext="png")

In [ ]:
from livecell_tracker.segment.ou_simulator import find_contours_opencv
test_mask = mask_dataset.get_img_by_time(31) == 1
plt.imshow(test_mask)
find_contours_opencv(test_mask)

In [ ]:
import livecell_tracker.core.io_sc
from livecell_tracker.core.io_sc import prep_scs_from_mask_dataset
import importlib
importlib.reload(livecell_tracker.core.io_sc)
scs = livecell_tracker.core.io_sc.prep_scs_from_mask_dataset(mask_dataset=mask_dataset, img_dataset=dic_dataset)

In [ ]:
import importlib
importlib.reload(livecell_tracker.core.single_cell)
scs_json_path = out_dir / "cellpose_raw_scs.json"
livecell_tracker.core.single_cell.SingleCellStatic.write_single_cells_json(scs, scs_json_path, dataset_dir=out_dir/"dataset")

Read single cell object data

In [ ]:
import json
from livecell_tracker.core.single_cell import SingleCellTrajectoryCollection, SingleCellStatic
from livecell_tracker.track.sort_tracker_utils import (
    track_SORT_bbox_from_scs
)

# all_scs_json_path = "./datasets/test_scs_EBSS_starvation/tmp_corrected_scs.json"
# all_scs_json_path = f"./datasets/test_scs_EBSS_starvation/{pos}/scs.json"
# all_scs = SingleCellStatic.load_single_cells_json(all_scs_json_path)
all_scs = SingleCellStatic.load_single_cells_json(scs_json_path)
sctc = track_SORT_bbox_from_scs(all_scs, raw_imgs=all_scs[0].img_dataset, min_hits=3, max_age=3)

In [ ]:
from livecell_tracker.core.sc_video_utils import gen_mp4_from_frames, gen_class2sample_samples, gen_samples_mp4s

# importlib reload import above
import importlib
importlib.reload(core.sc_video_utils)

In [ ]:
!pwd

In [ ]:
video_classify_output_dir = Path(f"./notebook_results/tmp_CFP_data_{pos}")

In [ ]:
from livecell_tracker.track.classify_utils import gen_one_sc_samples_by_window, gen_inference_sctc_sample_videos
sc_samples = []
samples_info_list = []


tid2samples = gen_one_sc_samples_by_window(sctc, window_size=7)
for tid, samples in tid2samples.items():
    for i, sample in enumerate(samples):
        sc_samples.append(sample)
        samples_info_list.append({"tid": tid, "sample_idx": i})

saved_sample_info_df = gen_inference_sctc_sample_videos(sctc, padding_pixels=[20, 50, 100], out_dir=video_classify_output_dir, prefix=f"{pos}", fps=3)

In [ ]:
saved_sample_info_df.to_csv(video_classify_output_dir / "sample_info.csv")

Read `saved_sample_info_df` if processed earlier

In [ ]:
import pandas as pd
saved_sample_info_dir = video_classify_output_dir / "sample_info.csv"
saved_sample_info_df = pd.read_csv(saved_sample_info_dir)

In [ ]:
video_classify_output_dir

In [ ]:
saved_sample_info_df[:2]

In [ ]:
mitosis_tids = set()
mitosis_rows = []
for i, row in tqdm(saved_sample_info_df.iterrows()):
    video_path = row["path"]
    video_path = str(video_classify_output_dir / "videos" / video_path)
    results = inference_recognizer(model, video_path)
    predicted_label = results.pred_labels.item.cpu().numpy()[0]
    if predicted_label == 0:
        print("mitosis found:", row["path"])
        mitosis_rows.append(row)
        mitosis_tids.add(row["track_id"])

mitosis_tids

In [ ]:
mitosis_rows[:2]

In [ ]:
# copy videos from paths from mitosis rows to a subfolder
import shutil
mitosis_video_dir = Path("notebook_results/tmp_mitosis_videos")
mitosis_video_dir.mkdir(exist_ok=True, parents=True)
for i, row in tqdm(enumerate(mitosis_rows)):
    filename = row["path"]
    video_path = str(video_classify_output_dir / "videos" / filename)
    shutil.copy(video_path, mitosis_video_dir / filename)



In [ ]:
from mmcv.video.io import VideoReader
import tempfile
import numpy as np

In [ ]:
from livecell_tracker.track.classify_utils import video_frames_and_masks_from_sample, combine_video_frames_and_masks

Visualize mitosis cells

In [ ]:
from livecell_tracker.core.sct_operator import create_scts_operator_viewer

# mitosis_tids = [7, 4, 1, 16] # EBSS less than 24h, xy1, by <tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v6-combined-clipL=2>
mitosis_tids = [tid for tid in track_id_to_sample_results if 0 in track_id_to_sample_results[tid]]
mitosis_tids

In [ ]:
from livecell_tracker.core.sct_operator import create_scts_operator_viewer, create_scs_edit_viewer

# scts_operator = create_scs_edit_viewer(single_cells, img_dataset = dic_dataset, time_span=(145, 155))
# # If you would like to start from sctc, you can use the following code
# scts_operator = create_scts_operator_viewer(sctc, img_dataset = dic_dataset) #, time_span=(1, 2))
# scts_operator = create_scts_operator_viewer(sctc.subset(mitosis_tids), img_dataset = dic_dataset) #, time_span=(1, 2))

In [ ]:
# results = inference_recognizer(model, r"D:\LiveCellTracker-dev\notebooks\notebook_results\mmaction_train_data_v5\videos\mitosis_2_combined_padding-40.mp4")
# results

Load test dataframe and evaluate.

In [ ]:
import pandas as pd
# video_dir = r"./notebook_results/mmaction_train_data_v5/videos"
# mmaction_data_tsv = r"./notebook_results/mmaction_train_data_v6/test_data_combined.txt"
# video_dir = r"./notebook_results/mmaction_train_data_v8/videos"
# mmaction_data_tsv = r"./notebook_results/mmaction_train_data_v8/mmaction_test_data_combined.txt"
# mmaction_data_tsv = r"./notebook_results/mmaction_train_data_v8/mmaction_train_data_combined.txt"
# video_dir = r"./notebook_results/mmaction_train_data_v9/videos"
# mmaction_data_tsv = r"./notebook_results/mmaction_train_data_v9/mmaction_test_data_combined.txt"
# video_dir = r"./notebook_results/mmaction_train_data_v10-st/videos"
# mmaction_data_tsv = r"./notebook_results/mmaction_train_data_v10-st/mmaction_test_data_combined.txt"
video_dir = r"./notebook_results/mmaction_train_data_v10-drop-div/videos"
mmaction_data_tsv = r"./notebook_results/mmaction_train_data_v10-drop-div/mmaction_test_data_combined.txt"

# df columns are path and label
mmaction_data_df = pd.read_csv(mmaction_data_tsv, sep=" ", header=None)
mmaction_data_df.columns = ["path", "label"]
mmaction_data_df[:2]

In [ ]:
from tqdm import tqdm
# test on all video in the df
video_paths = []
gt_labels = []
for i, row in tqdm(mmaction_data_df.iterrows()):
    video_path = os.path.join(video_dir, row["path"])
    video_paths.append(video_path)
    gt_labels.append(row["label"])

In [ ]:
gt2total = {}
gt2correct = {}
for i, video_path in tqdm(enumerate(video_paths)):
    results = inference_recognizer(model, video_path)
    predicted_label = results.pred_labels.item.cpu().numpy()[0]
    gt_label = gt_labels[i]
    if gt_label not in gt2total:
        gt2total[gt_label] = 0
        gt2correct[gt_label] = 0
    gt2total[gt_label] += 1
    if predicted_label != gt_labels[i]:
        print("wrong prediction:", video_path, "predicted_label:", predicted_label, "gt_label:", gt_labels[i])
    else:
        gt2correct[gt_label] += 1

for gt_label, total in gt2total.items():
    correct = gt2correct[gt_label]
    print("gt_label:", gt_label, "total:", total, "correct:", correct, "acc:", correct / total)